In [58]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import signal




In [59]:
income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_1.csv")

In [60]:
year_income =  pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")
year_income

,gnd,pctile,year,count,hh_inc,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
0,F,1,2001,398740,4.991094e+02,79.68207,79.89346,0.392838,0.385240
1,F,1,2002,411159,5.413069e+02,79.18359,79.41062,0.353981,0.346817
2,F,1,2003,420298,5.291391e+02,79.36441,79.57217,0.364781,0.359097
3,F,1,2004,427728,4.992995e+02,79.89841,80.09248,0.352019,0.347992
4,F,1,2005,433665,4.741454e+02,80.28360,80.46593,0.334612,0.330986
...,...,...,...,...,...,...,...,...,...
2795,M,100,2010,444908,1.915397e+06,88.76955,88.86678,0.524046,0.504820
2796,M,100,2011,441576,1.595813e+06,88.60163,88.71263,0.572685,0.552049
2797,M,100,2012,442401,1.828705e+06,88.12717,88.25696,0.632195,0.610853
2798,M,100,2013,439032,1.809494e+06,88.82390,88.93484,0.576337,0.556298


In [63]:
male_year =year_income[year_income["gnd"] == "M"] 
male_year[male_year["year"]==2001]["pctile"]

1400      1
1414      2
1428      3
1442      4
1456      5
       ... 
2730     96
2744     97
2758     98
2772     99
2786    100
Name: pctile, Length: 100, dtype: int64

In [107]:
def create_life_scatter(df):
    male_year =df[df["gnd"] == "M"] 
    female_year =df[df["gnd"] == "F"] 
    data = [go.Scatter(x=male_year["pctile"], y=signal.savgol_filter(male_year["le_agg"],
                           21, 3), line_color="blue", mode="lines", name="Male"),
            go.Scatter(x=male_year["pctile"], y=male_year["le_agg"], line=dict(color='rgba(5, 66, 163, 0.2)'), mode="markers", name="Male"),
            go.Scatter(x=female_year["pctile"], y=signal.savgol_filter(female_year["le_agg"],
                           21, 3), line_color="red", mode="lines", name="Female"),
            go.Scatter(x=female_year["pctile"], y=female_year["le_agg"], line=dict(color='rgba(196, 118, 128, 0.2)'), mode="markers", name="Female"),
            go.Scatter(x=[0, 100], y = [male_year["le_agg"].mean(), male_year["le_agg"].mean()], mode="lines",name="Male Avg", line_color="#7694c4", line_dash="dash"),
            go.Scatter(x=[0, 100], y = [female_year["le_agg"].mean(), female_year["le_agg"].mean()], mode="lines",name="Female Avg", line_color="#c47680", line_dash="dash")]
    return data




def compare_male_female():
    year_income =  pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")
    data = create_life_scatter(year_income[year_income["year"]==2001])
    
    layout = layout=go.Layout(
            #xaxis_type="log",
            title="Year: 2001",
            xaxis=dict(range=[0, 100], autorange=False),
            yaxis=dict(range=[65, 92], autorange=False),
            #margin=dict(l=0, r=0, b=0, t=50),


            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args =  [None, {"frame": {"duration": 1000, "redraw": False},
                                    "fromcurrent": True, 
                                    "transition": {"duration": 700,"easing": "quadratic-in-out"}
                                    }],)])])

    frames = []
    for year in year_income["year"].unique():
        frames.append(go.Frame(data = create_life_scatter(year_income[year_income["year"]==year])))
    fig =go.Figure(data=data, layout=layout, frames=frames)
    
    for i, frame in enumerate(fig.frames):
        frame['layout']['title'] = f"Year: {2001+i} "
        #redraw frame
    for button in fig.layout.updatemenus[0].buttons:
        button['args'][1]['frame']['redraw'] = True


    return fig

In [108]:
compare_male_female()

In [103]:
def compare_male_female():
    year_income =  pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")
    data = create_life_scatter(year_income[year_income["year"]==2001])
    
    layout = layout=go.Layout(
            #xaxis_type="log",
            title="Year: 2001",
            xaxis=dict(range=[0, 100], autorange=False),
            yaxis=dict(range=[65, 92], autorange=False),
            #margin=dict(l=0, r=0, b=0, t=50),


            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args =  [None, {"frame": {"duration": 1000, "redraw": False},
                                    "fromcurrent": True, 
                                    "transition": {"duration": 700,"easing": "quadratic-in-out"}
                                    }],)])])

    frames = []
    for year in year_income["year"].unique():
        frames.append(go.Frame(data = create_life_scatter(year_income[year_income["year"]==year])))
    fig =go.Figure(data=data, layout=layout, frames=frames)
    
    for i, frame in enumerate(fig.frames):
        frame['layout']['title'] = f"Year: {2001+i} "
        #redraw frame
    for button in fig.layout.updatemenus[0].buttons:
        button['args'][1]['frame']['redraw'] = True



    return fig
compare_male_female()

ValueError: 
    Invalid element(s) received for the 'steps' property of layout.slider
        Invalid elements include: [Frame({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Male',
              'type': 'scatter',
              'x': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
                           15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
                           29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
                           43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
                           57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
                           71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
                           85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
                           99, 100], dtype=int64),
            

In [104]:
male_year = year_income[year_income["gnd"] == "M"]
female_year = year_income[year_income["gnd"] == "F"]
male_year

,gnd,pctile,year,count,hh_inc,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
1400,M,1,2001,387717,4.286913e+02,72.87766,73.35743,0.308299,0.306622
1401,M,1,2002,398266,4.426572e+02,71.43374,71.93658,0.275866,0.275720
1402,M,1,2003,406375,4.187205e+02,71.53559,72.02927,0.283628,0.284918
1403,M,1,2004,412934,3.831861e+02,72.48225,72.95515,0.282825,0.281899
1404,M,1,2005,418273,3.634149e+02,72.08122,72.55158,0.281706,0.283001
...,...,...,...,...,...,...,...,...,...
2795,M,100,2010,444908,1.915397e+06,88.76955,88.86678,0.524046,0.504820
2796,M,100,2011,441576,1.595813e+06,88.60163,88.71263,0.572685,0.552049
2797,M,100,2012,442401,1.828705e+06,88.12717,88.25696,0.632195,0.610853
2798,M,100,2013,439032,1.809494e+06,88.82390,88.93484,0.576337,0.556298


In [61]:
year_income["count"].mean()

433773.84571428574

In [62]:
income_life_exp

,gnd,pctile,count,hh_inc,hh_inc_age40,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
0,F,1,7098335,4.327821e+02,3.833254e+02,78.52628,78.78384,0.052538,0.062370
1,F,2,7120033,1.919116e+03,1.681780e+03,80.29138,80.46241,0.056187,0.064146
2,F,3,7131115,3.718216e+03,3.344087e+03,81.04365,81.16810,0.056927,0.065437
3,F,4,7135812,5.466639e+03,5.051834e+03,81.43418,81.53068,0.058089,0.065256
4,F,5,7140055,7.089429e+03,6.671779e+03,81.54852,81.62885,0.058225,0.064854
...,...,...,...,...,...,...,...,...,...
195,M,96,6953071,2.570520e+05,2.249077e+05,86.21402,86.32386,0.075620,0.084606
196,M,97,6954378,2.972074e+05,2.557433e+05,86.40608,86.52405,0.077477,0.084408
197,M,98,6956188,3.654243e+05,3.064940e+05,86.64326,86.77003,0.078087,0.085943
198,M,99,6956929,5.116917e+05,4.125774e+05,86.77115,86.91302,0.074041,0.080363


In [63]:
male = income_life_exp[income_life_exp["gnd"]=="M"]
female = income_life_exp[income_life_exp["gnd"]=="F"]

In [64]:
male.head(10)

,gnd,pctile,count,hh_inc,hh_inc_age40,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
100,M,1,6767752,356.2361,284.1466,72.37481,72.74278,0.042176,0.057908
101,M,2,6796273,1602.8470,1198.6200,74.22428,74.49557,0.045743,0.056516
102,M,3,6810846,3222.1970,2463.7220,75.25354,75.45113,0.047029,0.055741
103,M,4,6819310,4904.6830,3878.3910,75.69355,75.83810,0.048603,0.055750
104,M,5,6823613,6551.4180,5331.4660,75.92894,76.03336,0.051366,0.056555
105,M,6,6825662,8128.4990,6765.5670,76.10738,76.18174,0.051604,0.054616
106,M,7,6829004,9651.2240,8171.8350,76.23241,76.28391,0.051851,0.055801
107,M,8,6828386,11102.8600,9488.6440,76.15797,76.19464,0.053153,0.055615
108,M,9,6831289,12531.0000,10751.0900,76.32673,76.34561,0.052309,0.054242
109,M,10,6834092,13934.4300,12055.4100,76.37934,76.38644,0.053013,0.055344


In [65]:
male["decile"] = pd.qcut(male["pctile"], 4, labels=False)
male_10 = male.groupby(by=["decile"]).mean().reset_index()

C:\Users\zeusg\AppData\Local\Temp/ipykernel_5500/1478791551.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [66]:
male_10

,decile,pctile,count,hh_inc,hh_inc_age40,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
0,0,13.0,6838237.88,17133.967764,15143.704064,76.662301,76.719770,0.052407,0.055300
1,1,38.0,6892188.64,47337.187600,43022.760800,80.174699,80.273496,0.062329,0.064368
2,2,63.0,6922383.92,83085.404000,76761.741200,82.640330,82.769159,0.071125,0.074272
3,3,88.0,6945433.28,255320.032000,212308.581600,85.187799,85.284052,0.075914,0.083457


In [67]:
top = male_10[(male_10["decile"]==0) | (male_10["decile"]==3)]
top

,decile,pctile,count,hh_inc,hh_inc_age40,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
0,0,13.0,6838237.88,17133.967764,15143.704064,76.662301,76.719770,0.052407,0.055300
3,3,88.0,6945433.28,255320.032000,212308.581600,85.187799,85.284052,0.075914,0.083457


In [68]:
male[male["pctile"]==1]["le_agg"].values[0]
male.pctile.max()

100

In [69]:
male["dummy"] ="US"
female["dummy"] ="US"

In [70]:
fig = go.Figure()
fig.add_trace(go.Violin(y=male["le_agg"], x = male["dummy"], line_color='blue', side = "positive", scalegroup='No', name='Male'))
fig.add_trace(go.Violin( y=female["le_agg"], x = female["dummy"], line_color='red', side = "positive", name='Female'))
#fig.add_vline(x=male[male["pctile"]==1]["le_agg"].values[0], line_width=3, line_dash="dash", line_color="green")
#fig.add_vline(x=male[male["pctile"]==25]["le_agg"].values[0], line_width=3, line_dash="dash", line_color="green")
#fig.add_vline(x=male[male["pctile"]==100]["le_agg"].values[0], line_width=3, line_dash="dash", line_color="green")
fig.update_traces(meanline_visible=True,
                  points='outliers', # show all points
                  jitter=0.1,  # add some jitter on points for better visibility
                  scalemode='count',

                  #change to overlaying violin plots
                  ) 
fig.update_layout(
    violingap=0, violingroupgap=0, violinmode='overlay')
fig.show()

In [71]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=male["le_agg"], histnorm="percent"))
fig.show()

In [72]:
fig = px.ecdf(income_life_exp, x="le_agg", color="gnd", markers=True, marginal="rug")
fig.show()

In [73]:
fig = go.Figure()
fig.add_trace(go.Violin(x=male["le_agg"], side="positive"))
fig.show()

In [74]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

x=df['day'][ df['smoker'] == 'No' ]
x

0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
       ... 
235     Sat
238     Sat
239     Sat
242     Sat
243    Thur
Name: day, Length: 151, dtype: object

In [75]:
fig = make_subplots(rows=2, cols=2, specs=[[{"type": "violin"}, {}],[{}, {"type": "violin"}]], 
                    row_heights=[1, 3], column_widths=[3, 1], shared_xaxes=True, shared_yaxes=True,
                    vertical_spacing = 0, horizontal_spacing=0)
fig.add_trace(go.Violin(x=male["hh_inc"], side="positive", name="hh_inc", line_color="red"), row=1, col=1)
fig.add_trace(go.Violin(x=female["hh_inc"], side="positive", name="hh_inc", line_color="blue"), row=1, col=1)
fig.add_trace(go.Violin(y=male["le_agg"], side="positive", name="le_agg", line_color="red"), row=2, col=2)
fig.add_trace(go.Violin(y=female["le_agg"], side="positive", name="le_agg", line_color="blue"), row=2, col=2)

fig.add_trace(go.Scatter(x=male["hh_inc"], y=male["le_agg"], mode='markers', name="men", line_color="red"), row=2, col=1)
fig.add_trace(go.Scatter(x=female["hh_inc"], y=female["le_agg"], mode='markers', name="women", line_color="blue"), row=2, col=1)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=50), showlegend=False)

#remove margin between subplots
fig.update_yaxes(automargin=True)
#remove background color for subplots

fig.update_layout( plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=0, b=0))
        

fig.show()

In [ ]:
def create_life_scatter(df):
    male_year =df[df["gnd"] == "M"] 
    female_year =df[df["gnd"] == "F"] 
    data = [go.Scatter(x=male_year["pctile"], y=signal.savgol_filter(male_year["le_agg"],
                           21, 3), line_color="blue", mode="lines", name="Male"),
            go.Scatter(x=male_year["pctile"], y=male_year["le_agg"], line=dict(color='rgba(5, 66, 163, 0.2)'), mode="markers", name="Male"),
            go.Scatter(x=female_year["pctile"], y=signal.savgol_filter(female_year["le_agg"],
                           21, 3), line_color="red", mode="lines", name="Female"),
            go.Scatter(x=female_year["pctile"], y=female_year["le_agg"], line=dict(color='rgba(196, 118, 128, 0.2)'), mode="markers", name="Female"),
            go.Scatter(x=[0, 100], y = [male_year["le_agg"].mean(), male_year["le_agg"].mean()], mode="lines",name="Male Avg", line_color="#7694c4", line_dash="dash"),
            go.Scatter(x=[0, 100], y = [female_year["le_agg"].mean(), female_year["le_agg"].mean()], mode="lines",name="Female Avg", line_color="#c47680", line_dash="dash")]
    
    
    
    fig.add_trace(go.Violin(x=male["hh_inc"], side="positive", name="hh_inc", line_color="red"), row=1, col=1)
    fig.add_trace(go.Violin(x=female["hh_inc"], side="positive", name="hh_inc", line_color="blue"), row=1, col=1)
    fig.add_trace(go.Violin(y=male["le_agg"], side="positive", name="le_agg", line_color="red"), row=2, col=2)
    fig.add_trace(go.Violin(y=female["le_agg"], side="positive", name="le_agg", line_color="blue"), row=2, col=2)
    fig.add_trace(go.Scatter(x=male["hh_inc"], y=male["le_agg"], mode='markers', name="men", line_color="red"), row=2, col=1)
    fig.add_trace(go.Scatter(x=female["hh_inc"], y=female["le_agg"], mode='markers', name="women", line_color="blue"), row=2, col=1)

    
    
    
    
    
    return data




def compare_male_female():
    year_income =  pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")
    data = create_life_scatter(year_income[year_income["year"]==2001])
    
    layout = layout=go.Layout(
            #xaxis_type="log",
            title="Year: 2001",
            xaxis=dict(range=[0, 100], autorange=False),
            yaxis=dict(range=[65, 92], autorange=False),
            #margin=dict(l=0, r=0, b=0, t=50),


            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args =  [None, {"frame": {"duration": 1000, "redraw": False},
                                    "fromcurrent": True, 
                                    "transition": {"duration": 700,"easing": "quadratic-in-out"}
                                    }],)])])

    frames = []
    for year in year_income["year"].unique():
        frames.append(go.Frame(data = create_life_scatter(year_income[year_income["year"]==year])))
    fig =go.Figure(data=data, layout=layout, frames=frames)
    
    for i, frame in enumerate(fig.frames):
        frame['layout']['title'] = f"Year: {2001+i} "
        #redraw frame
    for button in fig.layout.updatemenus[0].buttons:
        button['args'][1]['frame']['redraw'] = True


    return fig

In [76]:
fig = go.Figure()
fig.add_trace(go.Violin(x=male["hh_inc"], side="positive", name="hh_inc"))
fig.update_layout(xaxis_type="log")
fig.show()

In [77]:
state_income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_3.csv")
state_income_life_exp.head(3)
state_income_life_exp = state_income_life_exp.sort_values(by=["le_agg_q1_M"])
state_income_life_exp["male_diff"] = state_income_life_exp["le_agg_q4_M"] - state_income_life_exp["le_agg_q1_M"]

In [78]:

data = {"line_x": [], "line_y": []}
for state in state_income_life_exp["statename"]:
    df = state_income_life_exp[state_income_life_exp["statename"]==state]
    data["line_x"].append(df["le_agg_q1_M"].values[0])
    data["line_x"].append(df["le_agg_q4_M"].values[0])
    data["line_x"].append(None)
    data["line_y"].extend([state, state, None])


In [85]:
state_income_life_exp.sort_values(by=["male_diff"], inplace=True) 

In [86]:
fig = go.Figure()
fig.add_trace(go.Scatter( x = state_income_life_exp["le_agg_q1_M"], y=state_income_life_exp["statename"], mode="markers", name = "Bottom 25%"))
fig.add_trace(go.Scatter( x = state_income_life_exp["le_agg_q4_M"], y=state_income_life_exp["statename"], mode="markers", name = "Top 25%"))
fig.add_trace(go.Scatter( x = data["line_x"], y=data["line_y"], mode="lines", marker={"color":"grey"}, opacity=0.5,))
fig.show()

In [87]:
state_income_life_exp["statename"]

4               California
32                New York
11                  Hawaii
30              New Jersey
21           Massachusetts
2                  Arizona
43                   Texas
9                  Florida
31              New Mexico
28                  Nevada
26                 Montana
12                   Idaho
47              Washington
37                  Oregon
13                Illinois
45                 Vermont
39            Rhode Island
6              Connecticut
5                 Colorado
27                Nebraska
41            South Dakota
23               Minnesota
3                 Arkansas
48           West Virginia
1                   Alaska
15                    Iowa
19                   Maine
24             Mississippi
29           New Hampshire
44                    Utah
49               Wisconsin
18               Louisiana
34            North Dakota
36                Oklahoma
25                Missouri
20                Maryland
17                Kentucky
3

In [88]:
state_income_life_exp.sample(10)

,st,stateabbrv,statename,le_agg_q1_F,le_raceadj_q1_F,le_agg_q2_F,le_raceadj_q2_F,le_agg_q3_F,le_raceadj_q3_F,le_agg_q4_F,...,sd_le_raceadj_q4_M,count_q1_F,count_q2_F,count_q3_F,count_q4_F,count_q1_M,count_q2_M,count_q3_M,count_q4_M,male_diff
3,5,AR,Arkansas,80.59530,81.31077,83.67512,84.12365,85.25621,85.63734,87.32423,...,0.191635,2070245,1804408,1490854,1025154,1882283,1762111,1408915,949898,8.80795
31,35,NM,New Mexico,83.10046,81.37305,85.04272,83.91698,86.21242,85.47363,87.80968,...,0.202401,1424728,1221518,1028852,873048,1315534,1112327,979814,810131,7.77566
12,16,ID,Idaho,82.54647,82.72810,85.35745,85.56174,86.52596,86.75501,88.26997,...,0.254892,835679,926619,904895,626539,815117,968469,888522,587945,7.96957
11,15,HI,Hawaii,84.21431,80.95252,86.03200,82.58419,86.85604,83.28809,88.46320,...,0.586456,707725,832662,823927,780904,706804,800000,842325,736583,7.19802
35,39,OH,Ohio,80.75004,81.40862,83.79350,84.26096,85.44341,85.84738,87.36544,...,0.095278,6747499,8345333,7878183,5826033,6093828,8017818,7630253,5517912,9.99710
23,27,MN,Minnesota,82.38299,82.60098,84.94285,85.24011,86.80253,87.06370,88.19151,...,0.116176,2116131,3093751,3887814,3790139,2158814,3093717,3825397,3626220,8.77676
7,10,DE,Delaware,81.02888,81.68574,83.49125,84.00578,85.45802,85.88902,87.63049,...,0.238135,449706,546751,579557,608615,404807,482162,544101,576733,10.12356
38,42,PA,Pennsylvania,81.61652,82.04772,84.37278,84.76311,85.90401,86.24136,87.68841,...,0.087549,6786670,8259170,8934549,7841779,6288464,8152420,8568669,7398819,9.60706
42,47,TN,Tennessee,81.01271,81.74355,83.71841,84.20284,85.48244,85.89891,87.41526,...,0.124888,4234824,4131402,3591447,2803003,3902261,3898583,3423900,2659138,9.94040
17,21,KY,Kentucky,80.86897,81.41580,83.41316,83.88276,85.18180,85.56631,87.03225,...,0.153902,2773567,2726363,2532249,1805827,2605410,2651305,2463581,1719501,9.57484


In [91]:
fig = go.Figure()
fig.add_trace(go.Choropleth(
        locations = state_income_life_exp["stateabbrv"],
        z =state_income_life_exp["male_diff"],
        locationmode = 'USA-states',
        colorscale='Blues',
        colorbar_x=0,
        marker_line_color='white'
    ))
fig.show()

In [402]:
fig = go.Figure()
dummy = np.linspace(74,87,100)
fig.add_trace(go.Scatter( x = state_income_life_exp["le_agg_q1_M"], y=state_income_life_exp["le_agg_q4_M"], text=state_income_life_exp["statename"], mode="markers", name = "Bottom 25%"))
fig.add_trace(go.Scatter( x = dummy, y=dummy, mode="lines", marker={"color":"grey"}, opacity=0.5, name="1:1"))

#fig.update_xaxes(range=[74,87])
#fig.update_yaxes(range=[74,87])
fig.update_layout(xaxis_title="Bottom 25% Life Exp", yaxis_title="Top 25% Life Exp",  margin=dict(l=0, r=0, t=0, b=0))

In [437]:
fig = go.Figure()
fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q1_F"], side="positive", name=""))
fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q1_M"], side="negative", name=""))

fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q2_F"], side="positive", name="q2"))
fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q2_M"], side="negative", name="q2"))

fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q3_F"], side="positive", name="q3"))
fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q3_M"], side="negative", name="q3"))

fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q4_F"], side="positive", name="Top"))
fig.add_trace(go.Violin(y=state_income_life_exp["le_agg_q4_M"], side="negative", name="Top"))


fig.update_layout(xaxis_title="Bottom 25% Life Exp", yaxis_title="Top 25% Life Exp",  margin=dict(l=0, r=0, t=0, b=0))

#fig.add_trace(go.Violin(x=state_income_life_exp["le_agg_q1_M"], side="positive", name="Male"))
#fig.add_trace(go.Violin(x=state_income_life_exp["le_agg_q4_M"], side="negative", name="Male"))

In [398]:
fig = go.Figure()


array([74.        , 74.13131313, 74.26262626, 74.39393939, 74.52525253,
       74.65656566, 74.78787879, 74.91919192, 75.05050505, 75.18181818,
       75.31313131, 75.44444444, 75.57575758, 75.70707071, 75.83838384,
       75.96969697, 76.1010101 , 76.23232323, 76.36363636, 76.49494949,
       76.62626263, 76.75757576, 76.88888889, 77.02020202, 77.15151515,
       77.28282828, 77.41414141, 77.54545455, 77.67676768, 77.80808081,
       77.93939394, 78.07070707, 78.2020202 , 78.33333333, 78.46464646,
       78.5959596 , 78.72727273, 78.85858586, 78.98989899, 79.12121212,
       79.25252525, 79.38383838, 79.51515152, 79.64646465, 79.77777778,
       79.90909091, 80.04040404, 80.17171717, 80.3030303 , 80.43434343,
       80.56565657, 80.6969697 , 80.82828283, 80.95959596, 81.09090909,
       81.22222222, 81.35353535, 81.48484848, 81.61616162, 81.74747475,
       81.87878788, 82.01010101, 82.14141414, 82.27272727, 82.4040404 ,
       82.53535354, 82.66666667, 82.7979798 , 82.92929293, 83.06

In [ ]:
fig = px.area(x = state_income_life_exp["le_agg_q1_M"], y=)

In [114]:
state_income_life_exp = state_income_life_exp.sort_values(by=["male_diff"])
fig = go.Figure()
fig.add_trace(go.Bar(x=state_income_life_exp["statename"], y=state_income_life_exp["male_diff"]))

In [223]:
income_life_exp.sample(3)

,gnd,pctile,count,hh_inc,hh_inc_age40,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
21,F,22,7165714,25686.21,24357.77,83.00622,83.05891,0.070845,0.070048
125,M,26,6869276,33034.36,29851.44,78.52054,78.59075,0.058727,0.058935
128,M,29,6876965,36459.51,32999.11,79.01659,79.09267,0.059369,0.060343


In [224]:
year_income.sample(3)


,gnd,pctile,year,count,hh_inc,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
2767,M,98,2010,444894,365022.20,87.32317,87.46404,0.628349,0.609290
260,F,19,2009,461057,22952.16,83.86703,83.87455,0.538206,0.526602
572,F,41,2013,457394,43419.94,85.41750,85.42692,0.536674,0.526099


In [234]:
male_year.head(3)

,gnd,pctile,year,count,hh_inc,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
1400,M,1,2001,387717,428.6913,72.87766,73.35743,0.308299,0.306622
1401,M,1,2002,398266,442.6572,71.43374,71.93658,0.275866,0.275720
1402,M,1,2003,406375,418.7205,71.53559,72.02927,0.283628,0.284918


In [264]:
fig = go.Figure()
fig.add_trace(go.Violin(y=male["le_agg"], text=male["hh_inc"], name="All", side="positive", line_color="blue"))
fig.update_traces(hovertext = "<b><i>Sales</i>: %{y}</b>")
fig.update_layout(hoverlabel = dict(
    bgcolor = "green",
    font_size = 16,
    font_family = "Sitka Small"
))
fig.show()
#fig.add_trace(go.Violin(x=female["le_agg"], name="All", side="negative", line_color="red"))

In [1]:
male_year

NameError: name 'male_year' is not defined

In [326]:
data = [go.Violin(x=male_year[male_year["year"] == 2001]["le_agg"], side="positive", line_color="blue", name="All", meanline_visible=True),
        go.Violin(x=female_year[female_year["year"] == 2001]["le_agg"], side="positive", line_color="red", name="All", meanline_visible=True),
        ]
layout = layout=go.Layout(
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args =  [None, {"frame": {"duration": 500},
                                "fromcurrent": True, "transition": {"duration": 500,
                                                                    "easing": "linear"}}])])])
frames = []
for year in male_year["year"].unique():
    frames.append(go.Frame(data=[go.Violin(x=male_year[male_year["year"] == year]["le_agg"], side="positive", line_color="blue", meanline_visible=True),
                                 go.Violin(x=female_year[female_year["year"] == year]["le_agg"], side="positive", line_color="red", meanline_visible=True),]))
fig =go.Figure(data=data, layout=layout, frames=frames)
fig.show()

In [271]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=male["hh_inc"], y=male["le_agg"], name="men"))
fig.add_trace(go.Scatter(x=male["hh_inc"], y=female["le_agg"], name="women"))
fig.update_layout(xaxis_type="log")

In [300]:
male_year

,gnd,pctile,year,count,hh_inc,le_agg,le_raceadj,sd_le_agg,sd_le_raceadj
1400,M,1,2001,387717,4.286913e+02,72.87766,73.35743,0.308299,0.306622
1401,M,1,2002,398266,4.426572e+02,71.43374,71.93658,0.275866,0.275720
1402,M,1,2003,406375,4.187205e+02,71.53559,72.02927,0.283628,0.284918
1403,M,1,2004,412934,3.831861e+02,72.48225,72.95515,0.282825,0.281899
1404,M,1,2005,418273,3.634149e+02,72.08122,72.55158,0.281706,0.283001
...,...,...,...,...,...,...,...,...,...
2795,M,100,2010,444908,1.915397e+06,88.76955,88.86678,0.524046,0.504820
2796,M,100,2011,441576,1.595813e+06,88.60163,88.71263,0.572685,0.552049
2797,M,100,2012,442401,1.828705e+06,88.12717,88.25696,0.632195,0.610853
2798,M,100,2013,439032,1.809494e+06,88.82390,88.93484,0.576337,0.556298


In [473]:
data = [go.Scatter(x=male_year[male_year["year"] == 2001]["hh_inc"], y=male_year[male_year["year"] == 2001]["le_agg"], line_color="blue", mode='markers', name="men"),
        go.Scatter(x=female_year[female_year["year"] == 2001]["hh_inc"],y=female_year[female_year["year"] == 2001]["le_agg"], line_color="red", mode='markers', name="women"),
        ]
layout = layout=go.Layout(
        xaxis_type="log",
        title="Start Title",
        xaxis=dict(range=[2, 6], autorange=False),
        yaxis=dict(range=[65, 92], autorange=False),
        margin=dict(l=0, r=0, b=0, t=50),
        

        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args =  [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 500,
                                                                    "easing": "linear"}}],)])])

frames = []
for year in male_year["year"].unique():
    frames.append(go.Frame(data=[go.Scatter(x=male_year[male_year["year"] == year]["hh_inc"],y=male_year[male_year["year"] == year]["le_agg"], mode='markers', line_color="blue"),
                                 go.Scatter(x=female_year[female_year["year"] == year]["hh_inc"],y=female_year[female_year["year"] == year]["le_agg"], mode='markers', line_color="red"),]))
    
fig =go.Figure(data=data, layout=layout, frames=frames)

fig.show()

In [442]:
fig = go.Figure()
fig.add_trace(go.Violin(x=male["hh_inc"], side="positive", line_color="blue", name="All", meanline_visible=True))
fig.add_trace(go.Violin(x=female["hh_inc"], side="negative", line_color="red", name="All", meanline_visible=True))

In [447]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=male["hh_inc"], y=female["hh_inc"], mode="markers"))
dummy = np.linspace(0, male["hh_inc"].max(), 100)
fig.add_trace(go.Scatter(x=dummy, y=dummy, mode="lines"))
fig.update_layout(xaxis_type="log", yaxis_type="log")


In [454]:
diff = male["hh_inc"].reset_index()["hh_inc"] - female["hh_inc"].reset_index()["hh_inc"] 
diff

0       -76.546
1      -316.269
2      -496.019
3      -561.956
4      -538.011
        ...    
95    14705.900
96    18031.000
97    23551.900
98    34271.100
99    59026.000
Name: hh_inc, Length: 100, dtype: float64

In [456]:
diff.index.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64)

In [458]:
fig = go.Figure()
fig.add_trace(go.Bar(x=diff.index.values, y=diff))

In [92]:
state_income_life_exp

,st,stateabbrv,statename,le_agg_q1_F,le_raceadj_q1_F,le_agg_q2_F,le_raceadj_q2_F,le_agg_q3_F,le_raceadj_q3_F,le_agg_q4_F,...,sd_le_raceadj_q4_M,count_q1_F,count_q2_F,count_q3_F,count_q4_F,count_q1_M,count_q2_M,count_q3_M,count_q4_M,male_diff
4,6,CA,California,84.43626,82.94646,84.85812,83.53072,86.08279,84.96513,87.83971,...,0.130121,20215088,18383557,18250582,23014857,20578421,17762051,17285647,22682795,5.66308
32,36,NY,New York,83.64188,83.17820,84.59113,84.40704,85.93414,85.82339,87.82232,...,0.072093,11425079,11154236,10691480,12046317,10869961,10073625,9993580,11526314,6.79663
11,15,HI,Hawaii,84.21431,80.95252,86.03200,82.58419,86.85604,83.28809,88.46320,...,0.586456,707725,832662,823927,780904,706804,800000,842325,736583,7.19802
30,34,NJ,New Jersey,82.86314,82.62794,83.99551,83.89870,85.25310,85.17694,87.47746,...,0.095425,4096902,4680733,5409202,8207123,3849772,4243716,5039654,8029962,7.30080
21,25,MA,Massachusetts,83.20533,83.06703,84.44691,84.63467,86.10712,86.29142,87.89267,...,0.097563,2837249,3604240,4314331,5766711,2699548,3352023,4046247,5540157,7.66908
2,4,AZ,Arizona,82.64610,82.15131,84.39893,84.14719,85.87695,85.78168,87.41205,...,0.112572,3281760,3395651,3120125,3014634,3288830,3113884,2943030,2875894,7.68192
43,48,TX,Texas,82.31120,81.29330,83.94273,83.27802,85.44335,85.01746,87.47135,...,0.063149,14634030,12717054,11451458,12471034,13428559,12224750,11300635,12383342,7.73161
9,12,FL,Florida,83.12566,82.85593,84.63407,84.43063,86.05415,85.91119,87.67615,...,0.069177,13398561,11438087,9454997,8610905,12807322,10520680,8736183,8228656,7.74955
31,35,NM,New Mexico,83.10046,81.37305,85.04272,83.91698,86.21242,85.47363,87.80968,...,0.202401,1424728,1221518,1028852,873048,1315534,1112327,979814,810131,7.77566
28,32,NV,Nevada,80.66357,80.16355,82.57057,82.02397,84.06367,83.67476,86.09050,...,0.166965,1396072,1582808,1444143,1294757,1557991,1585093,1459025,1264011,7.85711


In [95]:
part = Partisian.get_state_part_score()

In [100]:
df = pd.merge(part, state_income_life_exp, left_on="State", right_on="stateabbrv")

In [103]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Part_Score, y=df.male_diff, mode ="markers"))